In [1]:
import requests
import pandas as pd
import urllib.request
import json

In [2]:
data = [1, 66, 999, 4.864, 5191.0]

In [2]:
# формируем запрос
def send_json(x):
    month, duration, pdays, euribor3m, nr_employed = x
    
    body = {'month': month, 
            'duration': duration,
            'pdays': pdays,
            'euribor3m': euribor3m,
            'nr_employed': nr_employed
            }
    myurl = 'http://127.0.0.1:5000/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [4]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

предсказание 0.0029015397449973254


In [3]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")
y_train = pd.read_csv("y_train.csv")

In [19]:
N = 150

In [20]:
%%time
predictions = X_test.iloc[:N,:].apply(lambda x: send_json(x),1)

Wall time: 10.2 s


In [15]:
predictions.values[:5]

array([0.01034679, 0.03249217, 0.3332292 , 0.02211452, 0.19101217])

In [7]:
from sklearn.metrics import roc_auc_score,roc_curve,scorer, precision_recall_curve
import numpy as np

C:\Users\YAShmarin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [27]:
y_train.value_counts()

y
0    27402
1     3486
dtype: int64

In [21]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9127798507462687

In [22]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values[:N], predictions.values)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.440739, F-Score=0.667, Precision=0.714, Recall=0.625
